In [ ]:
# download data, you can maunally download dataset set here:
# https://drive.google.com/drive/folders/1eP7FtPaWfJ5zLdcsZYl6eyn5EYixkFn8

!gdown --id 1JD3OaHpq_4KCb7ofcPMkknmdEXFHrCcn
!gdown --id 1ssRA7yijGLFmJU-ac-lPyUOq7DYzTAS1
!gdown --id 1Rpz-ZuQxDwvLyzc0FD9GZxAKlyka3VC5
!gdown --id 1ouMFNT1thia8l6P5vcWCY-nweAexLDsB
!gdown --id 1Xyo9voEtL8fFf0gh7NY8L0cC6z8mZDrl

/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1JD3OaHpq_4KCb7ofcPMkknmdEXFHrCcn
To: /content/hw3_mycocodata_mask_comp_zlib.h5
100% 4.30M/4.30M [00:00<00:00, 29.5MB/s]
/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1ssRA7yijGLFmJU-ac-lPyUOq7DYzTAS1
To: /content/hw3_mycocodata_labels_comp_zlib.npy
100% 269k/269k [00:00<00:00, 4.99MB/s]
/usr/local/lib/python3.10/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downl

In [ ]:
!pip install pytorch-lightning
!pip install torchtext
!pip install tensorboardX

In [ ]:
import os
import numpy as np
from scipy import ndimage

import torch
import torchvision
from torchvision import transforms, datasets
from torch import nn, optim
from matplotlib import pyplot as plt
from torch.utils import data
from torch.utils.data import DataLoader
import torch.nn.functional as F
import pytorch_lightning as pl
import pytorch_lightning.loggers as pl_loggers
import pytorch_lightning.callbacks as pl_callbacks

import pickle
import h5py

import cv2
from PIL import Image
from pathlib import Path
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from sklearn.model_selection import train_test_split
from sklearn.metrics import auc
import xml.etree.ElementTree as ET

import numpy as np
import torch
from solo_head import *
from backbone import *
from dataset import *
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data
from tensorboardX import SummaryWriter
import os.path
import gc
gc.enable()

root_dir = "/content/"

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print("device = ", device)

device =  cuda:0


In [ ]:
paths = ['hw3_mycocodata_img_comp_zlib.h5',
         'hw3_mycocodata_mask_comp_zlib.h5',
         'hw3_mycocodata_labels_comp_zlib.npy',
         'hw3_mycocodata_bboxes_comp_zlib.npy']

dataset = BuildDataset(paths)
del paths

In [ ]:
train_size = int(len(dataset) * 0.8)
test_size = len(dataset) - train_size

torch.random.manual_seed(1)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(0)

train_dataset, test_dataset = torch.utils.data.random_split(
    dataset, [train_size, test_size])

batch_size = 2
train_build_loader = BuildDataLoader(
    train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
train_loader = train_build_loader.loader()
test_build_loader = BuildDataLoader(
    test_dataset, batch_size=batch_size, shuffle=False, num_workers=0)
test_loader = test_build_loader.loader()

In [ ]:
resnet50_fpn = Resnet50Backbone()
solo_head = SOLOHead(num_classes=4)

resnet50_fpn = resnet50_fpn.to(device)
resnet50_fpn.eval()
solo_head = solo_head.to(device)

num_epochs = 10
optimizer = optim.SGD(solo_head.parameters(), lr=0.01 /
                      16*batch_size, momentum=0.9, weight_decay=0.0001)
scheduler = optim.lr_scheduler.MultiStepLR(
    optimizer, milestones=[27, 33], gamma=0.1)

In [ ]:
os.makedirs("logs", exist_ok=True)
writer = SummaryWriter(log_dir="logs")

os.makedirs("train_checkpt", exist_ok=True)

In [ ]:
# Train
train_losses = {"cate": [], "mask": [], "total": []}
test_losses = {"cate": [], "mask": [], "total": []}

for epoch in range(num_epochs):
    solo_head.train()
    cum_losses = {"cate": 0.0, "mask": 0.0, "total": 0.0}

    for iter, data in enumerate(train_loader):
        img, label_list, mask_list, bbox_list = data
        img, label_list, mask_list, bbox_list = img.to(device), [x.to(device) for x in label_list], [x.to(device) for x in mask_list], [x.to(device) for x in bbox_list]

        with torch.no_grad():
            backout = resnet50_fpn(img)

        fpn_feat_list = list(backout.values())
        optimizer.zero_grad()
        cate_pred_list, ins_pred_list = solo_head.forward(fpn_feat_list, eval=False)

        ins_gts_list, ins_ind_gts_list, cate_gts_list = solo_head.target(ins_pred_list, bbox_list, label_list, mask_list)
        cate_loss, mask_loss, total_loss = solo_head.loss(cate_pred_list, ins_pred_list, ins_gts_list, ins_ind_gts_list, cate_gts_list)

        total_loss.backward()
        optimizer.step()
        cum_losses["cate"] += cate_loss.item()
        cum_losses["mask"] += mask_loss.item()
        cum_losses["total"] += total_loss.item()

        if np.isnan(cum_losses["total"]):
            raise RuntimeError(f"[ERROR] NaN encountered - iter {iter}")

        if iter % 100 == 99:
            log_losses = {key: cum_losses[key] / 100.0 for key in cum_losses}
            for loss_type in ["cate", "mask", "total"]:
                writer.add_scalar(f'Loss/train/log_{loss_type}_loss', log_losses[loss_type], len(train_losses[loss_type]))
                train_losses[loss_type].append(log_losses[loss_type])
            print(f'\nIteration:{iter+1} Avg. train total loss: {log_losses["total"]:.4f}')
            cum_losses = {key: 0.0 for key in cum_losses}

    path = f'./train_checkpt/solo_epoch_{epoch}'
    torch.save({
        'epoch': epoch,
        'model_state_dict': solo_head.state_dict(),
        'optimizer_state_dict': optimizer.state_dict()
    }, path)

    # Validation
    solo_head.eval()
    test_cum_losses = {"cate": 0.0, "mask": 0.0, "total": 0.0}

    with torch.no_grad():
        for iter, data in enumerate(test_loader):
            img, label_list, mask_list, bbox_list = data
            img, label_list, mask_list, bbox_list = img.to(device), [x.to(device) for x in label_list], [x.to(device) for x in mask_list], [x.to(device) for x in bbox_list]

            backout = resnet50_fpn(img)
            fpn_feat_list = list(backout.values())
            cate_pred_list, ins_pred_list = solo_head.forward(fpn_feat_list, eval=False)

            ins_gts_list, ins_ind_gts_list, cate_gts_list = solo_head.target(ins_pred_list, bbox_list, label_list, mask_list)
            cate_loss, mask_loss, total_loss = solo_head.loss(cate_pred_list, ins_pred_list, ins_gts_list, ins_ind_gts_list, cate_gts_list)
            test_cum_losses["cate"] += cate_loss.item()
            test_cum_losses["mask"] += mask_loss.item()
            test_cum_losses["total"] += total_loss.item()

        epoch_losses = {key: test_cum_losses[key] / len(test_loader.dataset) for key in test_cum_losses}
        print(f'\nEpoch:{epoch + 1} Avg. test loss: {epoch_losses["total"]:.4f}')

        for loss_type in ["cate", "mask", "total"]:
            writer.add_scalar(f'Loss/test/{loss_type}_loss', epoch_losses[loss_type], epoch)
            test_losses[loss_type].append(epoch_losses[loss_type])

    scheduler.step()

writer.close()


In [ ]:
# Inference


class ModelEvaluator:
    def __init__(self, tracker_id):
        self.reset()
        self.tracker_id = tracker_id

    def reset(self):
        self.conf_scores = np.zeros((0,))
        self.tp_indicator = np.zeros((0,), np.bool_)
        self.match_idx = np.zeros((0,), np.int32)
        self.num_gts = 0
        self.precision_recall = dict()

    def copy(self):
        copy_instance = ModelEvaluator(self.tracker_id)
        copy_instance.conf_scores = self.conf_scores.copy()
        copy_instance.tp_indicator = self.tp_indicator.copy()
        copy_instance.match_idx = self.match_idx.copy()
        copy_instance.num_gts = self.num_gts
        copy_instance.precision_recall = self.precision_recall.copy()

        return copy_instance

    def update_metrics(self, conf_scores, tp_indicator, match_idx, num_gt_batch):
        conf_scores = conf_scores.detach().cpu().numpy() if isinstance(conf_scores, torch.Tensor) else conf_scores
        tp_indicator = tp_indicator.detach().cpu().numpy() if isinstance(tp_indicator, torch.Tensor) else tp_indicator
        match_idx = match_idx.detach().cpu().numpy() if isinstance(match_idx, torch.Tensor) else match_idx

        assert isinstance(conf_scores, np.ndarray)
        assert isinstance(tp_indicator, np.ndarray)
        assert isinstance(match_idx, np.ndarray)
        assert len(conf_scores) == len(tp_indicator)
        assert len(match_idx) == len(tp_indicator)

        if len(match_idx) == 0:
            self.num_gts += num_gt_batch
            return

        if num_gt_batch == 0:
            assert np.all(tp_indicator == False)
        else:
            assert np.max(match_idx) < num_gt_batch

        self.conf_scores = np.concatenate([self.conf_scores, conf_scores])
        self.tp_indicator = np.concatenate([self.tp_indicator, tp_indicator])
        match_idx_new = match_idx + self.num_gts

        if self.tracker_id == 0:
            debug = True

        self.match_idx = np.concatenate([self.match_idx, match_idx_new])
        self.num_gts += num_gt_batch

        for i in range(match_idx_new.shape[0]):
            if tp_indicator[i] and (match_idx_new[i] >= self.num_gts):
                raise RuntimeError("Check Failed")

    def compute(self):
        precision_recall = {}
        n_tp, n_tot_pred, num_gts = 0.0, 0.0, self.num_gts

        sorted_idx = np.argsort(self.conf_scores)
        sorted_scores, sorted_tp_indicator, sorted_match_idx = self.conf_scores[sorted_idx], self.tp_indicator[sorted_idx], self.match_idx[sorted_idx]

        for score, is_tp, match_idx in zip(sorted_scores, sorted_tp_indicator, sorted_match_idx):
            n_tot_pred += 1
            if is_tp:
                n_tp += 1
            precision = n_tp / n_tot_pred
            recall = np.sum(sorted_tp_indicator[:n_tot_pred]) / num_gts
            precision_recall[recall] = precision

        self.precision_recall = precision_recall

        del sorted_scores, sorted_tp_indicator, sorted_match_idx
        return precision_recall


    def sort_curve(self):
        assert len(self.precision_recall) != 0, "[ERROR] Please compute precision and recall before calling sort_curve"
        recall_sorted = sorted(list(self.precision_recall.keys()))
        precision_sorted = [self.precision_recall[k] for k in recall_sorted]
        return recall_sorted, precision_sorted

    def compute_average_precision(self):
        assert len(self.precision_recall) != 0, "[ERROR] Please compute precision and recall before calling compute_average_precision"
        recall_sorted = sorted(list(self.precision_recall.keys()))
        precision_sorted = [self.precision_recall[k] for k in recall_sorted]
        if len(recall_sorted) > 1:
            return auc(recall_sorted, precision_sorted)
        else:
            return 0.0

os.makedirs("output", exist_ok=True)
os.makedirs("infer_result", exist_ok=True)

eval_epoch = 20
VISUALIZATION = True
batch_size = 2
cate_thresh = 0.33

resnet50_fpn = Resnet50Backbone()
solo_head = SOLOHead(num_classes=4)

solo_head.postprocess_cfg['cate_thresh'] = cate_thresh
print(f"[INFO] Category Threshold: {cate_thresh}")

print(f"[INFO] eval epoch: {eval_epoch}")
checkpoint_file = f"./train_checkpt/solo_epoch_{eval_epoch}"
checkpoint = torch.load(checkpoint_file)
print("[INFO] Loaded checkpoint file successfully")
solo_head.load_state_dict(checkpoint['model_state_dict'])

solo_head = solo_head.to(device)
solo_head.eval()

color_list = ["jet", "ocean", "Spectral"]

model_evals = []
for i in range(solo_head.num_classes - 1):
    model_evals.append(ModelEvaluator(i))

with torch.no_grad():
    for iter, data in enumerate(tqdm(test_loader), 0):
        img, label_list, mask_list, bbox_list = [
            data[i] for i in range(len(data))]
        img = img.to(device)
        label_list = [x.to(device) for x in label_list]
        mask_list = [x.to(device) for x in mask_list]
        bbox_list = [x.to(device) for x in bbox_list]

        backout = resnet50_fpn(img)
        fpn_feat_list = list(backout.values())
        cate_pred_list, ins_pred_list = solo_head.forward(
            fpn_feat_list, eval=True)

        NMS_sorted_scores_list, NMS_sorted_cate_label_list, NMS_sorted_ins_list = solo_head.PostProcess(
            ins_pred_list, cate_pred_list, (img.shape[2], img.shape[3]))
        del ins_pred_list
        del cate_pred_list
        if (VISUALIZATION):
            solo_head.PlotInfer(NMS_sorted_scores_list, NMS_sorted_cate_label_list, NMS_sorted_ins_list,
                                color_list, img, iter)

        for sorted_scores, sorted_cate_label, sorted_ins, label_gt, mask_gt in zip(NMS_sorted_scores_list,
                                                                                   NMS_sorted_cate_label_list,
                                                                                   NMS_sorted_ins_list, label_list,
                                                                                   mask_list):
            assert torch.all(sorted_cate_label != 0)
            for label_x in range(solo_head.num_classes):
                if (label_x == 0):
                    continue
                idx_gt = (label_gt == label_x)
                idx_pred = (sorted_cate_label == label_x)

                N_gt = torch.sum(idx_gt).item()
                N_pred = torch.sum(idx_pred).item()
                if N_gt == 0:
                    if N_pred == 0:
                        continue
                    conf_scores = sorted_scores[idx_pred]
                    tp_indicator = np.zeros((N_pred, ), dtype=np.bool_)
                    match_idx = np.zeros((N_pred, ), dtype=np.int32)
                else:
                    if N_pred != 0:
                        conf_scores = sorted_scores[idx_pred]
                        cate_label = sorted_cate_label[idx_pred]
                        ins_pred = sorted_ins[idx_pred]
                        assert len(ins_pred) == N_pred

                        ins_gt = mask_gt[idx_gt]

                        ious = solo_head.MatrixIOU(ins_pred > solo_head.postprocess_cfg['ins_thresh'],
                                                   ins_gt > solo_head.postprocess_cfg['ins_thresh'])
                        assert ious.shape == (N_pred, N_gt)
                        max_ious, iou_max_idx = torch.max(ious, dim=1)
                        assert max_ious.shape[0] == N_pred
                        assert iou_max_idx.shape[0] == N_pred

                        tp_indicator = (
                            max_ious > solo_head.postprocess_cfg['IoU_thresh'])
                        match_idx = iou_max_idx
                    else:
                        conf_scores = torch.zeros((0,))
                        tp_indicator = torch.zeros((0,), dtype=torch.bool)
                        match_idx = torch.zeros((0,), dtype=torch.int)

                tmp = model_evals[label_x - 1]
                if isinstance(tmp, ModelEvaluator):
                    tmp.update_metrics(conf_scores, tp_indicator,
                                  match_idx, N_gt)
                    model_evals[label_x - 1] = tmp.copy()
                else:
                    print("tmp is not of type ModelEvaluator")

        output_filename = os.path.join("output", f"results_iter_{iter}.txt")
        with open(output_filename, 'w') as output_file:
            output_file.write(f"Results of iter {iter}\n")
            for i in range(solo_head.num_classes - 1):
                tmp = model_evals[i]
                if isinstance(tmp, ModelEvaluator):
                    tmp.compute()
                    if not tmp.precision_recall:
                        continue
                    recall, precision = tmp.sort_curve()
                    average_precision = tmp.compute_average_precision()
                    output_file.write(f"class_id: {i}. average_precision: {average_precision}\n")
                    tmp.reset()
                else:
                    print("tmp is not of type ModelEvaluator")

    fig, ax = plt.subplots(1)
    for i in range(solo_head.num_classes - 1):
        tmp = model_evals[i]
        if isinstance(tmp, ModelEvaluator):
            tmp.compute()
            recall, precision = tmp.sort_curve()

            plt.plot(recall, precision)

            average_precision = tmp.compute_average_precision()
            print(f"class_id: {i}. average_precision: {average_precision}")
            tmp.reset()

        else:
            print("tmp is not of type ModelEvaluator")


In [ ]:
import matplotlib.pyplot as plt

# Plot training losses
plt.figure(figsize=(10, 6))
plt.plot(train_losses["total"], label="Train Total Loss")
plt.plot(train_losses["cate"], label="Train Category Loss")
plt.plot(train_losses["mask"], label="Train Mask Loss")
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Training Losses')
plt.legend()
plt.show()

# Plot validation losses
plt.figure(figsize=(10, 6))
plt.plot(val_losses["total"], label="Validation Total Loss")
plt.plot(val_losses["cate"], label="Validation Category Loss")
plt.plot(val_losses["mask"], label="Validation Mask Loss")
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Validation Losses')
plt.legend()
plt.show()
